# Atividade: CNNs para Classificação

Neste notebook, iremos preparar nosso próprio dataset e treinar um modelo de classificação de imagens.

## Preparando os dados

Os dados desta atividade serão baixados da internet. Utilizaremos para isso buscadores comuns. Em seguida, dividiremos em treinamento e validação.

In [1]:
!pip install icrawler

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [58]:
import os
import shutil
import random
from icrawler.builtin import GoogleImageCrawler, BingImageCrawler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Adquirindo as Imagens

Utilizaremos o iCrawler para baixar imagens em buscadores através de termos especificados. Defina sua lista de classes.

In [13]:
def download_images(keyword, folder, n_total=100):
    os.makedirs(folder, exist_ok=True)
    downloaded = len(os.listdir(folder))
    remaining = n_total - downloaded

    while downloaded < n_total:
        crawler = BingImageCrawler(storage={'root_dir': folder})
        crawler.crawl(keyword=keyword, max_num=remaining, file_idx_offset=downloaded)
        downloaded = len(os.listdir(folder))
        remaining = n_total - downloaded
        print(f"Downloaded {downloaded}/{n_total}")

    print("Download complete!")

In [38]:
search_terms = {
    "vader": "darth vader", # nome da classe: termo que será usado na busca
    "yoda": "mestre yoda",
    "luke": "luke skywalker",
    "stormtrooper": "stormtrooper"
}

for label, term in search_terms.items():
    download_images(term, f"data/star_wars/{label}", n_total=100)

Download complete!
Downloaded 24/100


ERROR:downloader:Response status code 403, file https://i.redd.it/me-as-a-baby-mestre-ensinador-v0-9ymnnd51tj6a1.jpg


Downloaded 49/100


ERROR:downloader:Response status code 403, file https://i.redd.it/me-as-a-baby-mestre-ensinador-v0-9ymnnd51tj6a1.jpg


Downloaded 73/100


...

Downloaded 74/100
Downloaded 100/100
Download complete!


ERROR:downloader:Response status code 400, file https://media.istockphoto.com/id/864742274/photo/stormtrooper-portrait.jpg
ERROR:downloader:Response status code 403, file https://oyster.ignimgs.com/mediawiki/apis.ign.com/star-wars-episode-7/thumb/b/b9/Stormtrooper.jpg


Downloaded 41/100


ERROR:downloader:Response status code 400, file https://media.istockphoto.com/id/864742274/photo/stormtrooper-portrait.jpg
ERROR:downloader:Response status code 403, file https://oyster.ignimgs.com/mediawiki/apis.ign.com/star-wars-episode-7/thumb/b/b9/Stormtrooper.jpg


Downloaded 82/100
Downloaded 100/100
Download complete!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Treinamento e Validação

Dividiremos as imagens baixadas nas pastas `train` e `val`. Defina uma porcentagem.

In [39]:
def split_train_val(root_dir, train_ratio=0.7, seed=42):
    random.seed(seed)

    train_dir = root_dir + "_split/train"
    val_dir = root_dir + "_split/val"

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    for class_name in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        images = [os.path.join(class_path, f) for f in os.listdir(class_path)]
        images = [f for f in images if os.path.isfile(f)]
        random.shuffle(images)

        n_train = int(len(images) * train_ratio)

        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        for img in images[:n_train]:
            shutil.copy(img, os.path.join(train_class_dir, os.path.basename(img)))
        for img in images[n_train:]:
            shutil.copy(img, os.path.join(val_class_dir, os.path.basename(img)))

        print(f"{class_name}: {n_train} train, {len(images)-n_train} val")

In [40]:
split_train_val("data/star_wars", train_ratio=0.7, seed=42)

yoda: 70 train, 30 val
stormtrooper: 70 train, 30 val
vader: 70 train, 30 val
luke: 70 train, 30 val


## Dataset

Implemente um Dataset PyTorch que carregue as imagens baixadas com suas respectivas classes. Aplique data augmentation e carregue em batches.

In [48]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [55]:
dataset_path = "/content/data/star_wars_split"

dataset_train = datasets.ImageFolder(
    root=dataset_path + "/train",
    transform=train_transform
)

dataset_val = datasets.ImageFolder(
    root=dataset_path + "/val",
    transform=val_transform
)

dataloader_train = DataLoader(
    dataset_train,
    batch_size=16,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

dataloader_val = DataLoader(
    dataset_val,
    batch_size=16,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)


In [56]:
images, labels = next(iter(dataloader_train))
print("Treino:", images.shape, labels)

images, labels = next(iter(dataloader_val))
print("Validação:", images.shape, labels)


Treino: torch.Size([16, 3, 224, 224]) tensor([0, 2, 3, 2, 1, 0, 2, 3, 1, 2, 0, 1, 0, 3, 0, 2])
Validação: torch.Size([16, 3, 224, 224]) tensor([2, 2, 3, 2, 3, 2, 3, 3, 1, 2, 3, 3, 2, 1, 0, 2])


## Definição do Modelo

Defina aqui o modelo que será utilizado, sendo implementação própria ou um modelo pré-treinado. Teste diversas arquiteturas diferentes e verifique qual delas tem melhor desempenho em validação.

In [59]:
Model = nn.Sequential(
    # entrada com 3 canais (RGB)
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.MaxPool2d(2, 2),   # 224 → 112

    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.MaxPool2d(2, 2),   # 112 → 56

    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.MaxPool2d(2, 2),   # 56 → 28

    nn.Flatten(),  # (batch, 256 * 28 * 28)

    nn.Linear(256 * 28 * 28, 512),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(512, 4)  # classes: luke, stormtrooper, vader, yoda
)

## Treinamento

Defina a função de custo e o otimizador do modelo. Em seguida, implemente o código de treinamento e treine-o. Ao final, exiba as curvas de treinamento e validação para a loss e a acurácia.

In [ ]:
# Seu código aqui

## Inferência

Calcule algumas métricas como acurácia, matriz de confusão, etc. Em seguida, teste o modelo em novas imagens das classes correspondentes mas de outras fontes (outro buscador, fotos próprias, etc).

In [ ]:
# Seu código aqui

# Task
Create a PyTorch Dataset with data augmentation and batch loading using the images from `data/star_wars` directory, splitting them into training and validation sets. This includes correcting the data splitting call, defining data transformations (resize, random crop, horizontal flip, color jitter, normalization for training; resize, center crop, normalization for validation), implementing a custom `torch.utils.data.Dataset` class, and instantiating `torch.utils.data.DataLoader` for both training and validation sets.

## Corrigir divisão de dados

### Subtask:
Ajustar a chamada da função `split_train_val` para que ela processe o diretório correto onde as imagens foram baixadas (`data/star_wars`), em vez de um caminho inexistente.


**Reasoning**:
The previous execution of `split_train_val` failed due to a `FileNotFoundError` because the `root_dir` argument was incorrectly set to `'batch_data'`. The images were downloaded to `data/star_wars`, so the function call needs to be updated to reflect the correct directory to allow the data splitting to proceed successfully.

